In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

%autosave 50

Autosaving every 50 seconds


## Get user comments

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path, trunc=0):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1        
    if trunc > 0 and i > trunc: 
        break
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [4]:
# comments_df = getDF(data_path + file_name, 200000)
# comments_df.loc[0,'reviewText']
# print(comments_df.shape)
# comments_df.head(1)

In [5]:
# Save the records
# import pickle
# pickle_out = open(data_path + "amzn_200k.pickle","wb")
# pickle.dump(comments_df, pickle_out)
# pickle_out.close()

In [6]:
# Reads records
import pickle
pickle_in = open(data_path + "amzn_200k.pickle","rb")
comments_df = pickle.load(pickle_in)

## Split comments into separate sentences

In [206]:
small = comments_df.loc[:2, :]
len(small.index)

3

In [207]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [208]:
small.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime', 'sentence'],
      dtype='object')

In [209]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [210]:
small

,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,"[This has some great tips as always and is helping me to complete my Good Eats collection., I haven't tried any of the recipes yet, but I will soon., Sometimes it's just lovely to let Alton entertain us.]"
1,A3R5OBKS7OM2IR,0000143529,5.0,"[This is a great pastry guide., I love how Alton's collections can break it down so baking isn't so mystical and scary., I might even try some of these recipes some day.]"
2,AH3QC2PC1VTGP,0000143561,2.0,"[I have to admit that I am a fan of Giada's cooking and I had great expectations when I ordered this set., They were however, crushed., While I still love Giada's cooking, this set is just a way for Food Network to make money., They really cheated with these DVD's., All they have are the video from the show, no text recipes, no link to the on line shows and no computer support., They play in Windows media player but the set does not contain the recipes., You can get more by taping the shows and then going to the web to download recipes., Another disappointment is the so so transfer quality to DVD., Perhaps I've been spoiled by HD and Tivo but the older shows I've recorded have had better playback quality than the episodes on the DVD's., It is in the old 480p but the quality of the transfer to DVD is dark and about the same quality as your average old VHS tape, not DVD quality., I get the impression Food Network got cheap and subbed out the DVD transfer to the lowest bidder (China?), and it shows.I could watch Giada read the dictionary and her cooking is really first rate., But, that's all you get is watching Giada., Thank god she is easy to understand and her recipes are easy to follow., But to get consistent results on some of the dishes, you should search the web and find the hard copies., While Giada herself is great, this set is a waste of money., You're better off recording the shows and going to the web to get the recipes you want or, better yet, just stick with her cookbooks which are all first rate and worlds above this cheap presentation.]"


In [211]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(30)

(23, 4)


,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is helping me to complete my Good Eats collection.
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's cooking and I had great expectations when I ordered this set.
3,A3R5OBKS7OM2IR,0000143502,5.0,"I haven't tried any of the recipes yet, but I will soon."
4,A3R5OBKS7OM2IR,0000143529,5.0,I love how Alton's collections can break it down so baking isn't so mystical and scary.
5,AH3QC2PC1VTGP,0000143561,2.0,"They were however, crushed."
6,A3R5OBKS7OM2IR,0000143502,5.0,Sometimes it's just lovely to let Alton entertain us.
7,A3R5OBKS7OM2IR,0000143529,5.0,I might even try some of these recipes some day.
8,AH3QC2PC1VTGP,0000143561,2.0,"While I still love Giada's cooking, this set is just a way for Food Network to make money."
11,AH3QC2PC1VTGP,0000143561,2.0,They really cheated with these DVD's.


## Sentence-level prep & cleaning

In [212]:
%reload_ext autoreload
from utils import split_n_lower, not_about_support

### Split into words and lower the case

In [213]:
words = sentences['sentence'].apply(lambda s: split_n_lower(s))

In [214]:
print(words.shape)
words.head(2)

(23,)


0    [this, has, some, great, tips, as, always, and, is, helping, me, to, complete, my, good, eats, collection, .]
1    [this, is, a, great, pastry, guide, .]                                                                       
Name: sentence, dtype: object

### Remove support-related sentences

In [232]:
sentences_on_movie = sentences[[not_about_support(word) for word in words]]
print('Removing {} records'.format(sentences.shape[0]- sentences_on_movie.shape[0]))

Removing 5 records


### Remove objective sentences for case B using obj-subj model

In [230]:
# pd.set_option('display.max_colwidth', -1)
# sentences_on_movie['sentence']

In [231]:
sentences_on_movie.head(2)

,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is helping me to complete my Good Eats collection.
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.


#### Vectorize along the word space of the obj-subj training set

In [218]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = pickle.load(open('Obj-Subj_tfidf.pkl', 'rb'))
len(tfidf.vocabulary_)

20893

In [219]:
mat = tfidf.transform(sentences_on_movie['sentence']).todense()

In [220]:
mat

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

#### Apply the obj-subj model

In [221]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open('GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'rb')
gb_model = pickle.load(pickle_in)

In [222]:
y_test = gb_model.predict(mat)
y_test

array([1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
       0.])

In [223]:
subjective_sentences = sentences_on_movie[y_test == 1]

In [224]:
subjective_sentences

,reviewerID,asin,overall,sentence
0,A3R5OBKS7OM2IR,0000143502,5.0,This has some great tips as always and is helping me to complete my Good Eats collection.
1,A3R5OBKS7OM2IR,0000143529,5.0,This is a great pastry guide.
2,AH3QC2PC1VTGP,0000143561,2.0,I have to admit that I am a fan of Giada's cooking and I had great expectations when I ordered this set.
4,A3R5OBKS7OM2IR,0000143529,5.0,I love how Alton's collections can break it down so baking isn't so mystical and scary.
6,A3R5OBKS7OM2IR,0000143502,5.0,Sometimes it's just lovely to let Alton entertain us.
20,AH3QC2PC1VTGP,0000143561,2.0,You can get more by taping the shows and then going to the web to download recipes.
38,AH3QC2PC1VTGP,0000143561,2.0,"But, that's all you get is watching Giada."
44,AH3QC2PC1VTGP,0000143561,2.0,"But to get consistent results on some of the dishes, you should search the web and find the hard copies."
47,AH3QC2PC1VTGP,0000143561,2.0,"While Giada herself is great, this set is a waste of money."


#### Merge the sentences back into paragraph reviews

In [228]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])
subj_reviews_stars = subj_groups['overall'].mean()
# subjective_reviewssubjective_reviews['sentence'].apply(lambda x: x.sum())
# subjective_reviews_reviews = 
subj_reviews_stars

reviewerID      asin      
A3R5OBKS7OM2IR  0000143502    5.0
                0000143529    5.0
AH3QC2PC1VTGP   0000143561    2.0
Name: overall, dtype: float64

In [238]:
subj_review_comments = subj_groups['sentence'].sum()
print(subj_review_comments.shape)
print(subj_review_comments[0])
subj_review_comments

(3,)
This has some great tips as always and is helping me to complete my Good Eats collection.Sometimes it's just lovely to let Alton entertain us.


reviewerID      asin      
A3R5OBKS7OM2IR  0000143502    This has some great tips as always and is helping me to complete my Good Eats collection.Sometimes it's just lovely to let Alton entertain us.                                                                                                                                                                                                                                                          
                0000143529    This is a great pastry guide.I love how Alton's collections can break it down so baking isn't so mystical and scary.                                                                                                                                                                                                                                                                                    
AH3QC2PC1VTGP   0000143561    I have to admit that I am a fan of Giada's cooking and I had great expectations when I ordered th

### Numpy approach

In [80]:
sent_arr = sentences.to_numpy()
sent_arr[0]

array(['A3R5OBKS7OM2IR', '0000143502', 5.0,
       'This has some great tips as always and is helping me to complete my Good Eats collection.'],
      dtype=object)

In [81]:
sent_arr[:, 3]

array(['This has some great tips as always and is helping me to complete my Good Eats collection.',
       'This is a great pastry guide.',
       "I haven't tried any of the recipes yet, but I will soon.",
       "I love how Alton's collections can break it down so baking isn't so mystical and scary.",
       "Sometimes it's just lovely to let Alton entertain us.",
       'I might even try some of these recipes some day.'], dtype=object)

In [82]:
# words = [sent for sent in map(word_tokenize, sent_arr[0, 3])]
# words

In [83]:
# sentences['words'] = 
# sentences.apply(lambda row: print(type(row['sentence'])),axis=1) #
# sentences.apply(lambda row: ['one','two'],axis=1) #
sentences.apply(lambda row: word_tokenize(row['sentence']), axis=1)

0    [This, has, some, great, tips, as, always, and...
1               [This, is, a, great, pastry, guide, .]
2    [I, have, n't, tried, any, of, the, recipes, y...
3    [I, love, how, Alton, 's, collections, can, br...
4    [Sometimes, it, 's, just, lovely, to, let, Alt...
5    [I, might, even, try, some, of, these, recipes...
dtype: object

In [56]:
paragraph = "My mother drove me to the airport with the windows rolled down. It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue. I was wearing my favorite shirt – sleeveless, white eyelet lace; I was wearing it as a farewell gesture. My carry-on item was a parka. In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds. It rains on this inconsequential town more than any other place in the United States of America. It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old. It was in this town that I’d been compelled to spend a month every summer until I was fourteen. That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead."
paragraph
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

input_string = remove_accents(paragraph)
from nltk.tokenize import sent_tokenize

sent_tokens = np.array(sent_tokenize(input_string))

In [57]:
sent_tokens

array(['My mother drove me to the airport with the windows rolled down.',
       'It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue.',
       'I was wearing my favorite shirt  sleeveless, white eyelet lace; I was wearing it as a farewell gesture.',
       'My carry-on item was a parka.',
       'In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds.',
       'It rains on this inconsequential town more than any other place in the United States of America.',
       'It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old.',
       'It was in this town that Id been compelled to spend a month every summer until I was fourteen.',
       'That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead.'],
      dtype='<U144')

In [58]:
from nltk.tokenize import word_tokenize

tokens = [sent for sent in map(word_tokenize, sent_tokens)]
tokens

[['My',
  'mother',
  'drove',
  'me',
  'to',
  'the',
  'airport',
  'with',
  'the',
  'windows',
  'rolled',
  'down',
  '.'],
 ['It',
  'was',
  'seventy-five',
  'degrees',
  'in',
  'Phoenix',
  ',',
  'the',
  'sky',
  'a',
  'perfect',
  ',',
  'cloudless',
  'blue',
  '.'],
 ['I',
  'was',
  'wearing',
  'my',
  'favorite',
  'shirt',
  'sleeveless',
  ',',
  'white',
  'eyelet',
  'lace',
  ';',
  'I',
  'was',
  'wearing',
  'it',
  'as',
  'a',
  'farewell',
  'gesture',
  '.'],
 ['My', 'carry-on', 'item', 'was', 'a', 'parka', '.'],
 ['In',
  'the',
  'Olympic',
  'Peninsula',
  'of',
  'northwest',
  'Washington',
  'State',
  ',',
  'a',
  'small',
  'town',
  'named',
  'Forks',
  'exists',
  'under',
  'a',
  'near-constant',
  'cover',
  'of',
  'clouds',
  '.'],
 ['It',
  'rains',
  'on',
  'this',
  'inconsequential',
  'town',
  'more',
  'than',
  'any',
  'other',
  'place',
  'in',
  'the',
  'United',
  'States',
  'of',
  'America',
  '.'],
 ['It',
  'was',
  

## Create emotion vectors

In [54]:
small = comments_df.loc[:2000, :]
len(small.index)

2001

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
comments_emotions = emote.vectorize(small,'reviewText')

In [ ]:
comments_emotions.shape